### IMPORT LIBRARY

In [1]:
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
from office365.runtime.auth.client_credential import ClientCredential
from office365.runtime.client_request_exception import ClientRequestException
import datetime
import pytz
import pandas as pd
import numpy as np
import csv
import glob
import pyodbc
import os 
import json
from io import BytesIO
import io
import platform
from function.PyToSp import *
from tqdm import tqdm
from urllib.parse import quote_plus
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, event, text
import pyodbc
import requests
import inspect
from validate_update import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from urllib.parse import quote_plus
import msal
from itertools import chain
from termcolor import colored
from send_email import *

### CONNECT TO AZURE SQL

In [2]:
def ConnectAzureSQLServer(): 
    f = open ('database_information.json', "r") #Database information file, can change information depending on the time
    qq = json.loads(f.read())
    f.close()
    ini_cnt_str ='Driver={driver_str};Server=tcp:{servername},1433;database={database};Uid={username};Pwd={password};Encrypt=yes;Authentication=ActiveDirectoryPassword;Connection Timeout=30;'.format(**qq)
    quoted = quote_plus(ini_cnt_str)
    cnt_str = 'mssql+pyodbc:///?odbc_connect={}'.format(quoted)
    engine = create_engine(cnt_str)

        #Test Connection
    try:
        conn = engine.connect()
        result = conn.execute(text("SELECT 1"))
        print("CONNECTION SUCESSFUL!")
    except Exception as e:
        print("CONNECTION FAILED:",str(e))
    return cnt_str
cnt_str = ConnectAzureSQLServer()


CONNECTION SUCESSFUL!


### CONNECT TO SHAREPOINT

In [3]:
header_BIHub = 'share_point_BIHub'
config_BIHub = read_config_json(config_path, header_BIHub)
BIHub = SharePoint(config_BIHub)
BIHub.check_connect()

Web site title: S22M Research & BI Hub _ Successful Connection!


### READ AND LIST FOLDER

In [4]:
#Tất cả các tỉnh
relative_url = "/sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Fact"
sp_object = relative_url.split('/')[2].replace('-','')
list_folder = eval(sp_object).get_content_url(relative_url,return_list_folder=True)

Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Fact/Binh Duong Province
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Fact/Ha Noi City
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Fact/Ho Chi Minh City


In [5]:
#lấy năm
list_folder_sub1=[]
for i in list_folder:
    ls=eval(sp_object).get_content_url(i,return_list_folder=True)
    list_folder_sub1=list_folder_sub1+ls

Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Fact/Binh Duong Province/2023
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Fact/Ha Noi City/2022
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Fact/Ho Chi Minh City/2022
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Fact/Ho Chi Minh City/2023


In [6]:
#lấy quý
list_folder_sub2=[]
for i in list_folder_sub1:
    ls=eval(sp_object).get_content_url(i,return_list_folder=True)
    list_folder_sub2=list_folder_sub2+ls

Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Fact/Binh Duong Province/2023/Q1
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Fact/Ha Noi City/2022/Q4
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Fact/Ho Chi Minh City/2022/Q4
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Fact/Ho Chi Minh City/2023/Q1
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Fact/Ho Chi Minh City/2023/Q2


In [7]:
#Provide information
Province_name=['Ho Chi Minh City']#Chọn tỉnh cần import-- là tên folder 
Year=['2023']#Chọn năm cần import
Quarter=['Q2']#Chọn quý cần import
#Có thể chọn nhiều tỉnh, tên tỉnh để trong dấu nháy đơn và cách nhau bởi dấy phẩy. Áp dụng tương tự cho năm, quý
#-------------------------------------------------------
df_summ_file = pd.DataFrame({'Name':[],'ServerRelativeUrl':[], 'TimeLastModified':[], 'ModTime':[], 'Modified_by_ID':[]})
for i in list_folder_sub2:
    print(i)
    if i.split('/')[-3] in Province_name and i.split('/')[-2] in Year and i.split('/')[-1] in Quarter:
        df_summ_file = pd.concat([df_summ_file, eval(sp_object).get_content_url(i)])

list_url = df_summ_file['ServerRelativeUrl'].to_list()
#History file
df_query=pd.DataFrame(df_summ_file).reset_index(drop=True)
df_summ_file = df_summ_file.reset_index(drop=True)


/sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Fact/Binh Duong Province/2023/Q1
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Fact/Ha Noi City/2022/Q4
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Fact/Ho Chi Minh City/2022/Q4
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Fact/Ho Chi Minh City/2023/Q1
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Fact/Ho Chi Minh City/2023/Q2
Folder name: 
Files name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Fact/Ho Chi Minh City/2023/Q2/Update_VLTH_HCMC_20230630.xlsx
Files name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Fact/Ho Chi Minh City/2023/Q2/UPDATE_APT_HCMC_20230630.xlsx


### GET IMPORT FILE

In [8]:
url = []
for i in list_url:
    sector = i.split('/')[-1].split('_')[1].upper()
    if sector in ('VLTH'): #Chọn sector cần import: RETAIL, OFFICE, SA, HOTEL, APT, VLTH
        url.append(i)
    else:
        pass
url

['/sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Fact/Ho Chi Minh City/2023/Q2/Update_VLTH_HCMC_20230630.xlsx']

### IMPORT DATA

In [9]:
'''Prepare ingredients''' 
columns_that_need_unidecode = ['Sub_Project_Name', 'Project_City_Name', 'Project_District_Name']
#Create empty df for checking dictionary
df_dict = pd.DataFrame(columns=['File_Name', 'Missing_Values', 'Flag'])
name_sector = ['RETAIL', 'SA', 'APT', 'VLTH', 'OFFICE', 'HOTEL']
engine = create_engine(cnt_str)
for i in name_sector:
    globals()[f'df_update_{i.lower()}'] = pd.DataFrame([])
    globals()[f'df_flat__{i.lower()}'] = pd.DataFrame([])
#-------------------------------------------------------
'''Get data''' 
for file_url in tqdm(url):
    data, file_name, sector = get_data(relative_url, file_url)
    # #-------------------------------------------------------

    '''Validation step'''
    #Remove unfortmated values
    data = remove_unformated_character(data)

    #Remove unicode characters
    for i in columns_that_need_unidecode:
        data[i] = remove_unicode(data[i])
    #Check dictionary
    # lst_dict = ['City', 'District', 'Type', 'Grade']
    # lst_cls = ['Project_City_Name', 'Project_District_Name', 'Sub_Project_Type', 'Grade']
    lst_dict = ['City', 'District', 'Grade']
    lst_cls = ['Project_City_Name', 'Project_District_Name', 'Grade']
    for i, j in zip(lst_cls, lst_dict):
        data, df_dict = check_dictionary(df_dict, file_name, data, i, j, sector, engine, sp_object)
    print(df_dict)
    if len(df_dict) == 0:
        print(colored('Validate succesfully','green'))
        #-------------------------------------------------------
        '''Import data process'''
        #Check project key
        processed_data, flag_key = check_project_key(file_url, data, sector, engine)
        if flag_key == 0:
            data = get_project_key(flag_key, processed_data, data, sector, engine)
            if sector == 'RETAIL':
                df_update_retail = pd.concat([df_update_retail, data], axis=0)
                df_flat_retail = get_flat_file(df_update_retail, sector, engine)
                flag_update, data = update_flat_file(sector, df_flat_retail, df_update_retail)
            elif sector == 'OFFICE':
                df_update_office = pd.concat([df_update_office, data], axis=0)
                df_flat_office = get_flat_file(df_update_office, sector, engine)
                flag_update, data = update_flat_file(sector, df_flat_office, df_update_office)
            elif sector == 'HOTEL':
                df_update_hotel = pd.concat([df_update_hotel, data], axis=0)
                df_flat_hotel = get_flat_file(df_update_hotel, sector, engine)
                flag_update, data = update_flat_file(sector, df_flat_hotel, df_update_hotel)
            elif sector == 'SA' or sector=='SERVICED_APARTMENT':
                df_update_sa = pd.concat([df_update_sa, data], axis=0)
                df_flat_sa = get_flat_file(df_update_sa, sector, engine)
                flag_update, data = update_flat_file(sector, df_flat_sa, df_update_sa)
            elif sector == 'APT' or sector=='APARTMENT':
                df_update_apt = pd.concat([df_update_apt, data], axis=0)
                df_flat_apt = get_flat_file(df_update_apt, sector, engine)
                flag_update, data = update_flat_file(sector, df_flat_apt, df_update_apt)
            elif sector == 'VLTH':
                df_update_vlth = pd.concat([df_update_vlth, data], axis=0)
                df_flat_vlth = get_flat_file(df_update_vlth, sector, engine)
                flag_update, data = update_flat_file(sector, df_flat_vlth, df_update_vlth)
            #-------------------------------------------------------    
            if flag_update == 0:
                insert_to_fresh(file_url, data, cnt_str)
                list_df_update = [df_update_retail, df_update_office, df_update_hotel, df_update_sa, df_update_apt, df_update_vlth]
                for df_update in list_df_update:
                    if len(df_update) != 0:
                        df_update = Generate_Additional_Columns(df_update,url,df_summ_file,BIHub,engine,file_url)
                        insert_to_tracking(df_update, sector, engine)
            else:
                print(colored(f'Some keys wrong in {file_name}', 'yellow'))
                print(data)
        else:
            print(colored(f'Cant get key in {file_name}', 'yellow'))
    else:
        pass

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Empty DataFrame
Columns: [File_Name, Missing_Values, Flag]
Index: []
Validate succesfully
CONNECTION SUCESSFUL IN INSERT TO FRESH FUNCTION!
Insert Update_VLTH_HCMC_20230630 to Fresh
Insert Update_VLTH_HCMC_20230630 to Fresh
Insert Update_VLTH_HCMC_20230630 to Fresh


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:50<00:00, 50.35s/it]
